(w,)

progs (generic function with 1 method)

polys (generic function with 1 method)

exprs (generic function with 1 method)

In [5]:


function string2args(s::String)
    [(Int(c)-77)/10.0 for c in s]
end


string2args (generic function with 1 method)

In [6]:


function write_raw(iters, w, h)
    xrange = (minimum(iters["x"]), maximum(iters["x"]))
    xscale = (w-1) / (xrange[2]-xrange[1])
    yrange = (minimum(iters["y"]), maximum(iters["y"]))
    yscale = (h-1) / (yrange[2]-yrange[1])

    pixels = zeros(Int64, w, h)

    for i in 1:length(iters["x"])
        fx = iters["x"][i]
        fy = iters["y"][i]
        if abs(fx) == Inf || isnan(fx) || abs(fy) == Inf || isnan(fy)
            break
        end
        x = floor(Int64, 1+(fx - xrange[1]) * xscale)
        y = floor(Int64, 1+(fy - yrange[1]) * yscale)
        
        pixels[x, y] += 1
    end

    gmax = 1 / maximum(pixels)
    
    gs =  (pixels .* gmax) |> colorview(Gray)
    
    save("t.jpg", gs)
    gs
end

write_raw (generic function with 1 method)

In [7]:
function iterate(ps, states, ns)
    
    n = 1 + ns[end]-ns[1]

    iters = Dict{String, Vector{Float64}}()
    for d in keys(ps)
        iters[d] = zeros(Float64, n)
    end

    for i in 1:n
         Threads.@threads for d in keys(iters)
            iters[d][i] = ps[d](x=>states["x"], y=>states["y"], z=>states["z"], w=>states["w"])
        end
         Threads.@threads for d in keys(iters)
            states[d] = iters[d][i]
        end
        
    end

    iters
end

iterate (generic function with 1 method)

In [8]:
function lyup(vs)
    L = 0.
    for i in 2:length(vs)
        L += abs(vs[i-1] - vs[i])
    end
    L /= length(vs)
    0.721347 * log(L)
end

lyup (generic function with 1 method)

In [9]:
function valid(iters)
   for d in keys(iters)
        ld = lyup(iters[d])
        if isnan(ld) || ld == Inf
            return false
        end
        if lyup(iters[d][end-10:end]) < 0.5
            return false
        end
    end
    return true
end

valid (generic function with 1 method)

In [10]:
using Random 

global a

while true
    pgs, as = progs(2,2)
    a = rand(as)
    es = exprs(pgs)
    pl = polys(es)

    states = Dict{String, Float64}()
    states["x"] = 0.05
    states["y"] = 0.05
    states["z"] = 0.05
    states["w"] = 0.05

    iters = iterate(pl, states, 1:100)

    if valid(iters)
        
        states["x"] = 0.05
        states["y"] = 0.05
        states["z"] = 0.05
        states["w"] = 0.05

        write_raw(iterate(pl, states, 1:50000), 800, 600)
        break
    end
end


In [ ]:


println(pl)
dx = differentiate(pl["x"], x)
dy = differentiate(pl["y"], y)

for i in 1:100
    [dx(x=)]

In [10]:
Threads.nthreads()

1